# Validation, Exceptions, and Error Handling
*aka Developing Robust Code*



So far in our code, we have followed the "happy" path through our code where everything just works and we assume that our input values are correct.  If something went wrong, we either re-ran our code (e.g., bad input value from the user) or made a code fix and then re-ran the code.

Software needs to be reliable and robust. That means that it needs to prevent errors when possible, detect situations when errors do occur, and then recover from errors as appropriate.  

THroughout these notebooks, we have seen many errors and their associated messages - sometimes intentionally, sometimes accidentally, and sometimes just incorrect logic (semantic errors).


Now we need to make a few decisions:
1. How do we want to validate input?
2. How do we want to prevent errors?
3. How do we want to react and recover from errors?

There are no clear, fixed solutions for these questions.  As with many things with system design, the answers will depend upon the context.

For scripts that we write for ourselves, ignoring errors may be fine in some circumstances.  However, if for code or results that will be used by others, we do need to ensure our code performs in a robust manner.  

Input validation depends upon the source, potential harm of using the data unchecked, and the ability for the exception handling to detect and handle the message.  Input validation, while often times annoying and time-consuming, is the best way to produce robust code that has a minimal amount of security flaw.  DISCUSS THE OWASP TOP TEN

As we detect errors, we also need to determine who is responsible for recovering from the error?  Is it performed within the current routine / function?  or passed back up the caller stack?  How is the user informed?  How does this differ among command-line tools, local GUIs, and web applications.

## Revisiting User Input
In this below code snippet from the Iteration Notebook, the user is asked to enter grades until they are complete with a negative number.  We had already added some error checking to see if the user entered at least one grade before marking themselves complete.  However, what occurs if they enter a value that's not an integer?  Let's try ...

In [ ]:
total = 0
num_entries = 0

while True:
    grade = int(input("Enter a grade: "))
    if grade < 0:
        break
    total += grade
    num_entries += 1

if num_entries > 0:
    print("Average:",total/num_entries)
else:
    print("no grades entered")

Whether you entered a string literal or a float literal, you should have received a "ValueError" that occurs when Python attempts to convert the string return value from input into an integer.  

Another error that could have occurred if we were running this script from a shell session if "EOFError" if the input stream was closed (e.g., through the user typing ctrl+d).
